#  Solve a diagonal Sudoku

Simple notebook version of the first project of Udacity's Artificial Intelligence Nanodegree

Source: https://github.com/udacity/AIND-Sudoku

## Setup the diagonal Sudoku problem

In [12]:
rows = 'ABCDEFGHI'
cols = '123456789'


def cross(a, b):
    """
    Input:  a, b (string, tuple, list .. )
    Output: cross list of all elements of a combined with b.
            Example: a='AB', b = '12' -> ['A1', 'A2', 'B1', 'B2']
    """
    return [s + t for s in a for t in b]


boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
diagonal_units = [[r + c
                   for r, c in zip(rows, cols)], [r + c for r, c in zip(rows, cols[::-1])]]
# [['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9'],
# ['A9', 'B8', 'C7', 'D6', 'E5', 'F4', 'G3', 'H2', 'I1']]
unitlist = row_units + column_units + square_units + diagonal_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s], [])) - {s}) for s in boxes)

## Input

A String representing a sudoku grid. Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3' <br>
The function below translates the string into a useful dictionary grid variable

In [13]:
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Input:  grid(string) - A grid in string form.
    Output: A grid in dictionary form
                Keys: The boxes, e.g., 'A1'
                Values: The value in each box, e.g., '8'. 
                If the box has no value, then the value will be '123456789'.
    """
    assert len(grid) == 81, "Input grid must be a string of length 81 (9x9)"
    values = dict(zip(boxes, grid))

    # replace "." in the dictionary
    for key, value in values.items():
        if value == ".":
            value = "123456789"
        values[key] = value
    return values

In [14]:
sudoku = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
grid_sudoku = grid_values(sudoku)
print(grid_sudoku)

{'A1': '2', 'A2': '123456789', 'A3': '123456789', 'A4': '123456789', 'A5': '123456789', 'A6': '123456789', 'A7': '123456789', 'A8': '123456789', 'A9': '123456789', 'B1': '123456789', 'B2': '123456789', 'B3': '123456789', 'B4': '123456789', 'B5': '123456789', 'B6': '6', 'B7': '2', 'B8': '123456789', 'B9': '123456789', 'C1': '123456789', 'C2': '123456789', 'C3': '1', 'C4': '123456789', 'C5': '123456789', 'C6': '123456789', 'C7': '123456789', 'C8': '7', 'C9': '123456789', 'D1': '123456789', 'D2': '123456789', 'D3': '6', 'D4': '123456789', 'D5': '123456789', 'D6': '8', 'D7': '123456789', 'D8': '123456789', 'D9': '123456789', 'E1': '3', 'E2': '123456789', 'E3': '123456789', 'E4': '123456789', 'E5': '9', 'E6': '123456789', 'E7': '123456789', 'E8': '123456789', 'E9': '7', 'F1': '123456789', 'F2': '123456789', 'F3': '123456789', 'F4': '6', 'F5': '123456789', 'F6': '123456789', 'F7': '4', 'F8': '123456789', 'F9': '123456789', 'G1': '123456789', 'G2': '4', 'G3': '123456789', 'G4': '123456789', '

### Display Sudoku

In [15]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input:  values(dict): The sudoku in dictionary form
    """
    import copy
    val = copy.copy(values)
    for k, v in val.items():
        if len(v) > 1:
            val[k] = ""

    width = 1 + max(len(val[s]) for s in boxes)
    line = '+'.join(['-' * (width * 3)] * 3)
    for r in rows:
        print(''.join(val[r + c].center(width) + ('|' if c in '36' else '') for c in cols))
        if r in 'CF':
            print(line)

In [16]:
display(grid_sudoku)

2     |      |      
      |    6 |2     
    1 |      |  7   
------+------+------
    6 |    8 |      
3     |  9   |    7 
      |6     |4     
------+------+------
  4   |      |8     
    5 |2     |      
      |      |    3 


## Solving strategies 

In [17]:
def eliminate(values):
    """
    Go through all the boxes, and whenever there is a box with a value, eliminate this value
    from the values of all its peers.
    Input:  A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    # get solved boxes (1 value)
    solved = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved:
        for peer in peers[box]:
            # eliminate solved value from peers (if possible)
            value = values[peer].replace(values[box], '')
            values[peer] = value
    return values

In [18]:
def only_choice(values):
    """
    Go through all the units, and whenever there is a unit with a value that only fits in one 
    box, assign the value to this box.
    Input:  A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    for unit in unitlist:
        for digit in '123456789':
            # boxes per unit holding a particular digit
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                # Just 1 box has the digit in its values for this unit -> only choice
                values[dplaces[0]] = digit
    return values

In [19]:
def naked_twins(values):
    """
    Eliminate values using the naked twins strategy.
    Input: values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Output: the values dictionary with the naked twins eliminated from peers.
    """
    two_digit_boxes = [box for box in values.keys() if len(values[box]) == 2]
    # each two_digit_boxes searches for a twin among its peers
    for box in two_digit_boxes:
        for peer in peers[box]:
            if values[peer] == values[box]:
                # pair of naked twins found
                # getting its shared units:  usually one, although more is possible
                shared_units = [x for x in units[box] if x in units[peer]]
                # For each cell in each shared unit, remove the two digits of the naked twins
                # (except for the twins)
                for shared_unit in shared_units:
                    for b in shared_unit:
                        if b != box and b != peer:
                            for digit in values[box]:
                                value = values[b].replace(digit, '')
                                values[b] = value
    return values

## Reduce Sudoku

Main loop called from the seach strategy. It will try to solve the puzzle by calling the above strategies iteratively

In [20]:
def reduce_puzzle(values):
    """
    Iterate eliminate(), only_choice(), and naked_twins().
    If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of all the functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """

    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Eliminate Strategy
        values = eliminate(values)
        # Only Choice Strategy
        values = only_choice(values)
        # Naked Twins Strategy
        values = naked_twins(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

## Solve Sudoku using Depth-first search

In [21]:
def search(values):
    """
    Using depth-first search and propagation, try all possible values
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form or False if there is a box with zero 
            available values
    """
    # reduce the puzzle
    values = reduce_puzzle(values)
    if values is False:  # Unsolved (zero available values)
        return False
    if all(len(values[s]) == 1 for s in boxes):  # Solved
        return values

    # choose one of the unfilled squares with the fewest possibilities
    _, s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)

    # solve recursively each one of the resulting sudokus; if one returns a value (not False),
    # return the answer
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [22]:
def solve(string_sudoku):
    """
        Find the solution to a Sudoku grid.
        Input:  grid(string): a string representing a sudoku grid.
                Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52
                .............3'
        Output: The dictionary representation of the final sudoku grid. False if no solution.
    """

    grid_sudoku = grid_values(sudoku)

    print("\n\n  Sudoku:\n")
    display(grid_sudoku)

    solution = search(grid_sudoku)
    if solution:
        print("\n  Solution found: \n")
        display(solution)
    else:
        print("\n  Solution not found: \n")


# Add custom doagonal sudokus
sudokus = [
    '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3',
    '.2.8..493...392........5...3...........451...........6...5........127...251..9.3.'
]

for sudoku in sudokus:
    solve(sudoku)



  Sudoku:

2     |      |      
      |    6 |2     
    1 |      |  7   
------+------+------
    6 |    8 |      
3     |  9   |    7 
      |6     |4     
------+------+------
  4   |      |8     
    5 |2     |      
      |      |    3 

  Solution found: 

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


  Sudoku:

  2   |8     |4 9 3 
      |3 9 2 |      
      |    5 |      
------+------+------
3     |      |      
      |4 5 1 |      
      |      |    6 
------+------+------
      |5     |      
      |1 2 7 |      
2 5 1 |    9 |  3   

  Solution found: 

7 2 5 |8 1 6 |4 9 3 
4 8 6 |3 9 2 |5 1 7 
1 3 9 |7 4 5 |6 8 2 
------+------+------
3 7 4 |2 6 8 |9 5 1 
6 9 2 |4 5 1 |3 7 8 
5 1 8 |9 7 3 |2 4 6 
------+------+------
8 6 7 |5 3 4 |1 2 9 
9 4 3 |1 2 7 |8 6 5 
2 5 1 |6 8 9 |7 3 4 
